In [ ]:
import os
import sys
import numpy

sys.path.append('./lib/caffe/python/')
import caffe

In [ ]:
caffe.set_mode_gpu()

In [ ]:
image_net = caffe.Net('./lib/caffe/models/triplet_googlenet/triplet_googlenet_image_deploy.prototxt', 
                      './lib/caffe/models/triplet_googlenet/triplet_googlenet_finegrain.caffemodel',
                      caffe.TEST)

In [ ]:
transformer = caffe.io.Transformer({'data': numpy.shape(image_net.blobs['data'].data)})

In [ ]:
transformer.set_mean('data', numpy.load('./lib/caffe/python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2, 0, 1))
transformer.set_channel_swap('data', (2, 1, 0))
transformer.set_raw_scale('data', 255.0)

In [ ]:
images_path = './images/archive'
images = os.listdir(images_path)
print("Number of images: {}".format(len(images)))
print(images)

In [ ]:
features = []
for i, file_name in enumerate(images):
    if !filename.endswith(".png") && !filename.endswith(".jpeg"):
        continue
        
    print("Processing: {0}/{1}".format(i+1, len(images)))
    try:
        img = transformer.preprocess('data', caffe.io.load_image(images_path + "/" + file_name))
    except:
        print("Cannot load " + images_path + "/" + file_name + " as an image")
        continue
#     print(img.shape)
    img_reshaped = numpy.reshape(img, image_net.blobs['data'].data.shape)
#     print(img_reshaped.shape)
    features_image_all_layes = image_net.forward(data=img_reshaped)
#     print(features_image_all_layes)
    features_image_output_layer = numpy.copy(features_image_all_layes['pool5/7x7_s1_p'])
#     print(features_image_output_layer)
    features.append(features_image_output_layer)
# print(numpy.shape(features))
shape_features = numpy.shape(features)
features = numpy.reshape(features, ([shape_features[0], shape_features[2]]))
# print(features.shape)

In [ ]:
numpy.save('./data/images.npy', images)
numpy.save('./data/features.npy', features)

In [ ]:
numpy.load('./data/features.npy').shape